# Irene-run

## Run irene for Kr MC events

In [ ]:
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
from __future__ import print_function
import sys
import os
from glob import glob
from time import time

In [ ]:
import argparse

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np
import math

In [ ]:
from invisible_cities.cities.irene import Irene

In [ ]:
from   invisible_cities.cities.irene import Irene, S12Params as S12P, IRENE

In [ ]:
import invisible_cities.core.configure as conf

In [ ]:
from invisible_cities.core.system_of_units_c import SystemOfUnits

In [ ]:
units = SystemOfUnits()

### Configuration file

In [ ]:
CONF_file = os.environ['IC_CONFIG'] + '/irene_KrMC.conf'

In [ ]:
CONF_file

In [ ]:
%less /Users/jjgomezcadenas/Projects/ICCONF/irene_KrMC.conf

In [ ]:
IRENE(['IRENE', '-c', CONF_file])